In [1]:
import pandas as pd
import os
import numpy as np
from dateutil.relativedelta import relativedelta

In [2]:
# Load data
shelter_data_to_load = "Animal_Shelter_Intake_and_Outcome.csv"
shelter_data_df = pd.read_csv(shelter_data_to_load)
shelter_data_df.head()

,Name,Type,Breed,Color,Sex,Size,Date Of Birth,Impound Number,Kennel Number,Animal ID,...,Intake Subtype,Outcome Type,Outcome Subtype,Intake Condition,Outcome Condition,Intake Jurisdiction,Outcome Jurisdiction,Outcome Zip Code,Location,Count
0,NaN,CAT,SIAMESE,BLACK/BLACK,Female,KITTN,NaN,K22-042204,CS05,A412797,...,FIELD,TRANSFER,FORGOTTEN,TREATABLE/REHAB,TREATABLE/REHAB,SANTA ROSA,SANTA ROSA,95403.0,"95403(38.486997, -122.749134)",1
1,NaN,CAT,DOMESTIC SH,GRAY/WHITE,Male,KITTN,06/24/2022,K22-042194,CA03,A412786,...,PHONE,NaN,NaN,UNKNOWN,NaN,COUNTY,NaN,NaN,NaN,1
2,NaN,CAT,SIAMESE,GRAY/BLACK,Female,KITTN,NaN,K22-042201,CS05,A412794,...,FIELD,TRANSFER,FORGOTTEN,TREATABLE/REHAB,TREATABLE/REHAB,SANTA ROSA,SANTA ROSA,95403.0,"95403(38.486997, -122.749134)",1
3,NaN,CAT,SIAMESE,GRAY/BLACK,Female,KITTN,NaN,K22-042202,CS05,A412795,...,FIELD,TRANSFER,FORGOTTEN,TREATABLE/REHAB,TREATABLE/REHAB,SANTA ROSA,SANTA ROSA,95403.0,"95403(38.486997, -122.749134)",1
4,NaN,CAT,DOMESTIC SH,BLACK,Female,SMALL,12/30/2020,K22-041790,SMM,A412222,...,FIELD,TRANSFER,WE CARE,UNKNOWN,HEALTHY,COUNTY,OUT OF COUNTY,94574.0,"94574(38.525993, -122.432854)",1


In [3]:
# Data analysis
col_det_df = shelter_data_df
columns_details_df = pd.DataFrame()
count = col_det_df.count()
null = col_det_df.isnull().sum()
unique = col_det_df.nunique()
dtype = col_det_df.dtypes
columns_details_df['count'] = count
columns_details_df['null'] = null
columns_details_df['unique'] = unique
columns_details_df['dtype'] = dtype
columns_details_df

,count,null,unique,dtype
Name,17291,6179,6320,object
Type,23470,0,3,object
Breed,23470,0,1005,object
Color,23470,0,332,object
Sex,23470,0,5,object
Size,23437,33,7,object
Date Of Birth,17618,5852,5442,object
Impound Number,23470,0,23470,object
Kennel Number,23462,8,161,object
Animal ID,23470,0,21200,object


In [4]:
# Animal type analysis
type_count = shelter_data_df.groupby(["Type"]).count()
type_count

,Name,Breed,Color,Sex,Size,Date Of Birth,Impound Number,Kennel Number,Animal ID,Intake Date,...,Intake Subtype,Outcome Type,Outcome Subtype,Intake Condition,Outcome Condition,Intake Jurisdiction,Outcome Jurisdiction,Outcome Zip Code,Location,Count
Type,,,,,,,,,,,,,,,,,,,,,
CAT,4822,8330,8330,8330,8328,6066,8330,8327,8330,8330,...,8330,8268,8177,8330,8150,8330,6797,6797,6797,8330
DOG,11874,13242,13242,13242,13211,10958,13242,13237,13242,13242,...,13242,13140,13029,13242,13035,13242,11994,11934,11934,13242
OTHER,595,1898,1898,1898,1898,594,1898,1898,1898,1898,...,1898,1872,1767,1898,1758,1898,1384,1371,1371,1898


In [5]:
# Data cleaning pre-processing for ML
# Filter columns
filtered_df = shelter_data_df[['Type','Sex','Size','Date Of Birth','Intake Date',
                               'Days in Shelter','Intake Type','Outcome Type',
                               'Intake Condition','Intake Jurisdiction']]
# Filter only 'CAT' and 'DOG'
filtered_df = filtered_df[filtered_df['Type'] != "OTHER"]
# Change column name
filtered_df = filtered_df.rename(columns={'Type': 'Animal Type'})
# Group ML target classes
filtered_df['Outcome Type'].replace({'ADOPTION':'ADOPTION RETURN','RETURN TO OWNER':'ADOPTION RETURN'},inplace=True)
# Drop NaN
filtered_df = filtered_df.dropna()
# Change dates dtype
filtered_df[['Date Of Birth','Intake Date']] = filtered_df[['Date Of Birth','Intake Date']].apply(pd.to_datetime)
# Calculate 'Age at intake time' and drop
filtered_df['Age at Intake'] = filtered_df['Intake Date'] - filtered_df['Date Of Birth']
filtered_df['Age at Intake'] = filtered_df['Age at Intake']/np.timedelta64(1, 'Y')
# Remove negative values in 'Age at intake time'
filtered_df = filtered_df[filtered_df['Age at Intake'] > 0]
# Change 'Days in shelter' dtype and format
filtered_df['Days in Shelter'] = filtered_df['Days in Shelter'].str.replace(',', '').astype(float)
# drop days in shelter = 0
filtered_df = filtered_df[filtered_df['Days in Shelter'] > 0]
filtered_df.head()

,Animal Type,Sex,Size,Date Of Birth,Intake Date,Days in Shelter,Intake Type,Outcome Type,Intake Condition,Intake Jurisdiction,Age at Intake
4,CAT,Female,SMALL,2020-12-30,2022-06-30,41.0,STRAY,TRANSFER,UNKNOWN,COUNTY,1.497635
6,CAT,Male,SMALL,2015-08-03,2022-08-03,6.0,STRAY,TRANSFER,UNKNOWN,SANTA ROSA,7.000828
13,DOG,Male,MED,2021-10-11,2022-04-11,120.0,QUARANTINE,ADOPTION RETURN,UNKNOWN,SANTA ROSA,0.498299
14,DOG,Male,SMALL,2021-08-29,2022-08-04,5.0,CONFISCATE,ADOPTION RETURN,UNKNOWN,SANTA ROSA,0.930888
21,DOG,Male,MED,2021-03-26,2022-06-25,4.0,STRAY,ADOPTION RETURN,UNKNOWN,SANTA ROSA,1.248486


In [6]:
# Drop dates columns
filtered_df = filtered_df.drop('Date Of Birth', axis=1)
filtered_df = filtered_df.drop('Intake Date', axis=1)
filtered_df.head()

,Animal Type,Sex,Size,Days in Shelter,Intake Type,Outcome Type,Intake Condition,Intake Jurisdiction,Age at Intake
4,CAT,Female,SMALL,41.0,STRAY,TRANSFER,UNKNOWN,COUNTY,1.497635
6,CAT,Male,SMALL,6.0,STRAY,TRANSFER,UNKNOWN,SANTA ROSA,7.000828
13,DOG,Male,MED,120.0,QUARANTINE,ADOPTION RETURN,UNKNOWN,SANTA ROSA,0.498299
14,DOG,Male,SMALL,5.0,CONFISCATE,ADOPTION RETURN,UNKNOWN,SANTA ROSA,0.930888
21,DOG,Male,MED,4.0,STRAY,ADOPTION RETURN,UNKNOWN,SANTA ROSA,1.248486


In [7]:
# Data analysis
col_det_df = filtered_df
cleaned_columns_details_df = pd.DataFrame()
count = col_det_df.count()
null = col_det_df.isnull().sum()
unique = col_det_df.nunique()
dtype = col_det_df.dtypes
cleaned_columns_details_df['count'] = count
cleaned_columns_details_df['null'] = null
cleaned_columns_details_df['unique'] = unique
cleaned_columns_details_df['dtype'] = dtype
cleaned_columns_details_df

,count,null,unique,dtype
Animal Type,14355,0,2,object
Sex,14355,0,5,object
Size,14355,0,7,object
Days in Shelter,14355,0,249,float64
Intake Type,14355,0,6,object
Outcome Type,14355,0,7,object
Intake Condition,14355,0,5,object
Intake Jurisdiction,14355,0,13,object
Age at Intake,14355,0,3004,float64


In [8]:
# Generate a categorical variable list
filtered_df_cat = filtered_df.dtypes[filtered_df.dtypes == "object"].index.tolist()
filtered_df_cat

['Animal Type',
 'Sex',
 'Size',
 'Intake Type',
 'Outcome Type',
 'Intake Condition',
 'Intake Jurisdiction']

In [9]:
# Check the number of unique values in each column
filtered_df[filtered_df_cat].nunique()

Animal Type             2
Sex                     5
Size                    7
Intake Type             6
Outcome Type            7
Intake Condition        5
Intake Jurisdiction    13
dtype: int64

In [10]:
from sklearn.preprocessing import OneHotEncoder
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(filtered_df[filtered_df_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(filtered_df_cat)
encode_df.head()

,Animal Type_CAT,Animal Type_DOG,Sex_Female,Sex_Male,Sex_Neutered,Sex_Spayed,Sex_Unknown,Size_KITTN,Size_LARGE,Size_MED,...,Intake Jurisdiction_*PETALUMA,Intake Jurisdiction_*ROHNERT PARK,Intake Jurisdiction_*SEBASTOPOL,Intake Jurisdiction_*SONOMA,Intake Jurisdiction_*TRIBAL RESV,Intake Jurisdiction_*WINDSOR,Intake Jurisdiction_COUNTY,Intake Jurisdiction_OUT OF COUNTY,Intake Jurisdiction_SANTA ROSA,Intake Jurisdiction_UNKNOWN
0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
# Merge one-hot encoded features and drop the originals
filtered_df = filtered_df.merge(encode_df,left_index=True, right_index=True)
filtered_encoded_df = filtered_df.drop(filtered_df_cat,1)
filtered_encoded_df.head()

C:\Users\fernanm\Anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,Days in Shelter,Age at Intake,Animal Type_CAT,Animal Type_DOG,Sex_Female,Sex_Male,Sex_Neutered,Sex_Spayed,Sex_Unknown,Size_KITTN,...,Intake Jurisdiction_*PETALUMA,Intake Jurisdiction_*ROHNERT PARK,Intake Jurisdiction_*SEBASTOPOL,Intake Jurisdiction_*SONOMA,Intake Jurisdiction_*TRIBAL RESV,Intake Jurisdiction_*WINDSOR,Intake Jurisdiction_COUNTY,Intake Jurisdiction_OUT OF COUNTY,Intake Jurisdiction_SANTA ROSA,Intake Jurisdiction_UNKNOWN
4,41.0,1.497635,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6,6.0,7.000828,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
13,120.0,0.498299,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
14,5.0,0.930888,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
21,4.0,1.248486,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [12]:
filtered_encoded_df.columns

Index(['Days in Shelter', 'Age at Intake', 'Animal Type_CAT',
       'Animal Type_DOG', 'Sex_Female', 'Sex_Male', 'Sex_Neutered',
       'Sex_Spayed', 'Sex_Unknown', 'Size_KITTN', 'Size_LARGE', 'Size_MED',
       'Size_PUPPY', 'Size_SMALL', 'Size_TOY', 'Size_X-LRG',
       'Intake Type_ADOPTION RETURN', 'Intake Type_CONFISCATE',
       'Intake Type_OWNER SURRENDER', 'Intake Type_QUARANTINE',
       'Intake Type_STRAY', 'Intake Type_TRANSFER',
       'Outcome Type_ADOPTION RETURN', 'Outcome Type_DIED',
       'Outcome Type_DISPOSAL', 'Outcome Type_ESCAPED/STOLEN',
       'Outcome Type_EUTHANIZE', 'Outcome Type_RTOS', 'Outcome Type_TRANSFER',
       'Intake Condition_HEALTHY', 'Intake Condition_TREATABLE/MANAGEABLE',
       'Intake Condition_TREATABLE/REHAB', 'Intake Condition_UNKNOWN',
       'Intake Condition_UNTREATABLE', 'Intake Jurisdiction_*CLOVERDALE',
       'Intake Jurisdiction_*COTATI', 'Intake Jurisdiction_*HEALDSBURG',
       'Intake Jurisdiction_*PETALUMA', 'Intake Jurisdict

In [13]:
# Drop other outcome types
filtered_encoded_df = filtered_encoded_df.drop(['Outcome Type_DIED','Outcome Type_DISPOSAL',
                                                'Outcome Type_ESCAPED/STOLEN','Outcome Type_EUTHANIZE',
                                                'Outcome Type_RTOS','Outcome Type_TRANSFER'], axis=1)

In [14]:
# Copy with desired columns
scale_df = filtered_encoded_df[['Days in Shelter', 'Age at Intake']].copy()
scale_df.head()

,Days in Shelter,Age at Intake
4,41.0,1.497635
6,6.0,7.000828
13,120.0,0.498299
14,5.0,0.930888
21,4.0,1.248486


In [15]:
# Create the StandardScaler instance
from sklearn.preprocessing import RobustScaler
robust_scaler = RobustScaler()

In [16]:
# Fit the StandardScaler
robust_scaler.fit(scale_df)

RobustScaler()

In [17]:
# Scale the data
robust_scaled_data = robust_scaler.transform(scale_df)

In [18]:
# Create a DataFrame with the scaled data
transformed_scaled_data = pd.DataFrame(robust_scaled_data, index=scale_df.index, columns=scale_df.columns)
transformed_scaled_data.head()

,Days in Shelter,Age at Intake
4,1.08,-0.099075
6,-0.32,1.100567
13,4.24,-0.316920
14,-0.36,-0.222620
21,-0.40,-0.153387


In [19]:
filtered_encoded_scaled_df = filtered_encoded_df.merge(transformed_scaled_data, how = 'outer', 
                                                left_index = True, right_index = True)
filtered_encoded_scaled_df.head()

,Days in Shelter_x,Age at Intake_x,Animal Type_CAT,Animal Type_DOG,Sex_Female,Sex_Male,Sex_Neutered,Sex_Spayed,Sex_Unknown,Size_KITTN,...,Intake Jurisdiction_*SEBASTOPOL,Intake Jurisdiction_*SONOMA,Intake Jurisdiction_*TRIBAL RESV,Intake Jurisdiction_*WINDSOR,Intake Jurisdiction_COUNTY,Intake Jurisdiction_OUT OF COUNTY,Intake Jurisdiction_SANTA ROSA,Intake Jurisdiction_UNKNOWN,Days in Shelter_y,Age at Intake_y
4,41.0,1.497635,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.08,-0.099075
6,6.0,7.000828,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.32,1.100567
13,120.0,0.498299,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.24,-0.316920
14,5.0,0.930888,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.36,-0.222620
21,4.0,1.248486,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.40,-0.153387


In [20]:
filtered_encoded_scaled_df = filtered_encoded_scaled_df.drop(['Days in Shelter_x','Age at Intake_x'], axis=1)
filtered_encoded_scaled_df.head()

,Animal Type_CAT,Animal Type_DOG,Sex_Female,Sex_Male,Sex_Neutered,Sex_Spayed,Sex_Unknown,Size_KITTN,Size_LARGE,Size_MED,...,Intake Jurisdiction_*SEBASTOPOL,Intake Jurisdiction_*SONOMA,Intake Jurisdiction_*TRIBAL RESV,Intake Jurisdiction_*WINDSOR,Intake Jurisdiction_COUNTY,Intake Jurisdiction_OUT OF COUNTY,Intake Jurisdiction_SANTA ROSA,Intake Jurisdiction_UNKNOWN,Days in Shelter_y,Age at Intake_y
4,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.08,-0.099075
6,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.32,1.100567
13,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.24,-0.316920
14,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.36,-0.222620
21,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.40,-0.153387


In [23]:
filtered_encoded_scaled_df.to_csv('sonoma_shelter_ML_test.csv',index=False)